In [108]:
import os
import json
import math
import numpy as np
import tensorflow as tf
import torch

import grid2op
from grid2op.Reward import *
from grid2op.Action import *
from grid2op.Runner import Runner
from grid2op.Reward import L2RPNSandBoxScore, L2RPNReward
from grid2op.Agent import DoNothingAgent

%run d3qn.ipynb
%run opponent.ipynb

In [98]:
def train_d3qn_against_random_opponent(env, agent, opponent, num_pre_training_steps, n_iter,
                                       save_path, log_path):
    # Make sure we can fill the experience buffer
    if num_pre_training_steps < agent.batch_size * agent.num_frames:
        num_pre_training_steps = agent.batch_size * agent.num_frames
        
    # Loop vars
    num_training_steps = n_iter
    num_steps = num_pre_training_steps + num_training_steps
    step = 0
    alive_steps = 0
    total_reward, total_reward_opp = 0, 0
    agent.done = True
    print(f"Total number of steps: {num_steps}")

    # Create file system related vars
    logpath = os.path.join(log_path, agent.name)
    os.makedirs(save_path, exist_ok=True)
    modelpath = os.path.join(save_path, agent.name + ".h5")
    agent.tf_writer = tf.summary.create_file_writer(logpath, name=agent.name)
    agent._save_hyperparameters(save_path, env, num_steps)
    
    while step < num_steps:
        # Init first time or new episode
        if agent.done:
            new_obs = env.reset() # This shouldn't raise
            agent.reset(new_obs)
            opponent.reset()
        if cfg.VERBOSE and step % 1000 == 0:
            print("Step [{}] -- Random [{}]".format(step, agent.epsilon))

        # Save current observation to stacking buffer
        agent._save_current_frame(agent.state)

        # Choose an action
        if step <= num_pre_training_steps:
            a = agent.Qmain.random_move()
        elif np.random.rand(1) < agent.epsilon:
            a = agent.Qmain.random_move()
        elif len(agent.frames) < agent.num_frames:
            a = 0 # Do nothing
        else:
            a, _ = agent.Qmain.predict_move(np.array(agent.frames))

        # Convert it to a valid action
        act = agent.convert_act(a)
        # Execute action
        new_obs, reward, agent.done, info = env.step(act)
        new_state = agent.convert_obs(new_obs)
        if info["is_illegal"] or info["is_ambiguous"] or \
           info["is_dispatching_illegal"] or info["is_illegal_reco"]:
            if cfg.VERBOSE:
                print (a, info)
        total_reward += reward

        # Save new observation to stacking buffer
        agent._save_next_frame(new_state)

        # Save to experience buffer
        if len(agent.frames2) == agent.num_frames:
            agent.per_buffer.add(np.array(agent.frames),
                                a, reward,
                                np.array(agent.frames2),
                                agent.done)

        # Perform training when we have enough experience in buffer
        if step >= num_pre_training_steps:
            training_step = step - num_pre_training_steps
            # Decay chance of random action
            agent.epsilon = agent._adaptive_epsilon_decay(training_step)

            # Perform training at given frequency
            if step % cfg.UPDATE_FREQ == 0 and \
               len(agent.per_buffer) >= agent.batch_size:
                # Perform training
                agent._batch_train(training_step, step)

                if cfg.UPDATE_TARGET_SOFT_TAU > 0.0:
                    tau = cfg.UPDATE_TARGET_SOFT_TAU
                    # Update target network towards primary network
                    agent.Qmain.update_target_soft(agent.Qtarget.model, tau)

            # Every UPDATE_TARGET_HARD_FREQ trainings, update target completely
            if cfg.UPDATE_TARGET_HARD_FREQ > 0 and \
               step % (cfg.UPDATE_FREQ * cfg.UPDATE_TARGET_HARD_FREQ) == 0:
                agent.Qmain.update_target_hard(agent.Qtarget.model)
        
        ######## Opponent #########
        if not agent.done:
            attack = opponent.attack(new_obs)
            if attack is not None:
                print('ATTACK step {}: disconnected {}'.format(step, attack.as_dict()['set_line_status']['disconnected_id'][0]))
                attack_obs, reward_opp, agent.done, info = env.step(attack)
                if info["is_illegal"] or info["is_ambiguous"] or \
                   info["is_dispatching_illegal"] or info["is_illegal_reco"]:
                    if cfg.VERBOSE:
                        print(attack, info)
                total_reward_opp += reward_opp
                new_obs = attack_obs
                opponent.tell_attack_continues(None, None, None, None)
        
        if agent.done:
            agent.epoch_rewards.append(total_reward)
            agent.epoch_alive.append(alive_steps)
            if cfg.VERBOSE and step > num_pre_training_steps:
                print("step {}: Survived [{}] steps".format(step, alive_steps))
                print("Total reward [{}]".format(total_reward))
                print("Total reward opponent [{}]".format(total_reward_opp))
            alive_steps = 0
            total_reward = 0
            total_reward_opp = 0            
        else:
            alive_steps += 1
            
        ######## After Each Step #######
        if step > 0 and step % 2000 == 0: # save network every 5000 iters
            agent.save(modelpath)
        step += 1
        # Make new obs the current obs
        agent.obs = new_obs
        agent.state = new_state

    # Save model after all steps
    agent.save(modelpath)

In [99]:
# agent params
num_pre_training_steps = 256
learning_rate = 1e-4
initial_epsilon = 0.99
final_epsilon = 0.01
decay_epsilon = 20000
# opponent params
attack_period = 20 # 12 * 24
attack_lines = ['0_1_0', '0_4_1', '11_12_13', '12_13_14', '1_2_2', '1_3_3', '1_4_4', '2_3_5', '3_4_6', '3_6_15', '3_8_16', '4_5_17', '5_10_7', '5_11_8', '5_12_9', '6_7_18', '6_8_19', '8_13_11', '8_9_10', '9_10_12']

# training params
n_iter = 20000
env_name = "rte_case14_realistic"
# env = make(env_name, reward_class=IllegalBadReward)
env = make(env_name, reward_class=CombinedScaledReward)

# Register custom reward for training
cr = env._reward_helper.template_reward
#cr.addReward("overflow", CloseToOverflowReward(), 1.0)
cr.addReward("game", GameplayReward(), 1.0)
#cr.addReward("recolines", LinesReconnectedReward(), 1.0)
cr.addReward("l2rpn", L2RPNReward(), 2.0/float(env.n_line))
# Initialize custom rewards
cr.initialize(env)
# Set reward range to something managable
cr.set_range(-1.0, 1.0)

agent_name = "DDDQN"
save_path = "saved_agent_DDDQN_random_opponent_{}_{}".format(attack_period, n_iter)
log_path="tf_logs_DDDQN"

opponent_name = "Random"

agent = DoubleDuelingDQN(env.observation_space, env.action_space, name=agent_name,
                         is_training=True, learning_rate=learning_rate,
                         initial_epsilon=initial_epsilon, final_epsilon=final_epsilon, decay_epsilon=decay_epsilon)
opponent = RandomOpponent(env.observation_space, env.action_space, lines_attacked=attack_lines,
                          name=opponent_name, attack_period=attack_period)

train_d3qn_against_random_opponent(env, agent, opponent, num_pre_training_steps, n_iter,
                                   save_path, log_path)

Agent action size: 141
Total number of steps: 20256
Step [0] -- Random [0.99]
ATTACK step 1: disconnected 6
ATTACK step 91: disconnected 18
ATTACK step 196: disconnected 17
ATTACK step 225: disconnected 15
ATTACK step 240: disconnected 2
ATTACK step 245: disconnected 1
11 {'disc_lines': array([False, False, False, False, False, False, False, False, False,
        True, False,  True, False, False,  True, False,  True,  True,
       False, False]), 'is_illegal': True, 'is_ambiguous': False, 'is_dispatching_illegal': False, 'is_illegal_reco': False, 'opponent_attack_line': None, 'opponent_attack_sub': None, 'opponent_attack_duration': 0, 'exception': [Grid2OpException IllegalAction IllegalAction('Powerline with ids [10] have been modified illegally (cooldown)',), Grid2OpException DivergingPowerFlow DivergingPowerFlow('GAME OVER: Powerflow has diverged during computation or a load has been disconnected or a generator has been disconnected.',)], 'rewards': {}}
ATTACK step 257: disconnected 

step 472: Survived [1] steps
Total reward [-0.15654075145721436]
Total reward opponent [0]
step 476: Survived [3] steps
Total reward [1.1406135559082031]
Total reward opponent [0]
ATTACK step 478: disconnected 18
step 478: Survived [1] steps
Total reward [1.465042233467102]
Total reward opponent [-1.0]
step 479: Survived [0] steps
Total reward [-1.0]
Total reward opponent [0]
step 482: Survived [2] steps
Total reward [0.5158997774124146]
Total reward opponent [0]
step 488: Survived [5] steps
Total reward [2.8533897399902344]
Total reward opponent [0]
step 490: Survived [1] steps
Total reward [-0.2076730728149414]
Total reward opponent [0]
step 492: Survived [1] steps
Total reward [-0.448319673538208]
Total reward opponent [0]
step 493: Survived [0] steps
Total reward [-1.0]
Total reward opponent [0]
step 496: Survived [2] steps
Total reward [0.45682477951049805]
Total reward opponent [0]
ATTACK step 498: disconnected 0
step 501: Survived [4] steps
Total reward [2.2263232469558716]
Tota

step 711: Survived [0] steps
Total reward [-1.0]
Total reward opponent [0]
step 715: Survived [3] steps
Total reward [1.3093748092651367]
Total reward opponent [0]
step 716: Survived [0] steps
Total reward [-1.0]
Total reward opponent [0]
step 717: Survived [0] steps
Total reward [-1.0]
Total reward opponent [0]
step 719: Survived [1] steps
Total reward [-0.26376330852508545]
Total reward opponent [0]
step 721: Survived [1] steps
Total reward [-0.20497393608093262]
Total reward opponent [0]
step 724: Survived [2] steps
Total reward [0.5100200176239014]
Total reward opponent [0]
step 727: Survived [2] steps
Total reward [0.44681787490844727]
Total reward opponent [0]
loss = 19.07829
step 730: Survived [2] steps
Total reward [0.45963096618652344]
Total reward opponent [0]
step 734: Survived [3] steps
Total reward [1.0749626159667969]
Total reward opponent [0]
step 737: Survived [2] steps
Total reward [0.33296382427215576]
Total reward opponent [0]
ATTACK step 738: disconnected 6
step 739

step 987: Survived [0] steps
Total reward [-1.0]
Total reward opponent [0]
step 993: Survived [5] steps
Total reward [2.4450786113739014]
Total reward opponent [0]
step 995: Survived [1] steps
Total reward [-0.2667757272720337]
Total reward opponent [0]
step 999: Survived [3] steps
Total reward [1.3453102111816406]
Total reward opponent [0]
Step [1000] -- Random [0.8949981880496173]
step 1001: Survived [1] steps
Total reward [-0.19566822052001953]
Total reward opponent [0]
ATTACK step 1002: disconnected 18
step 1002: Survived [0] steps
Total reward [0.6514813899993896]
Total reward opponent [-1.0]
step 1005: Survived [2] steps
Total reward [0.30945849418640137]
Total reward opponent [0]
step 1006: Survived [0] steps
Total reward [-1.0]
Total reward opponent [0]
loss = 8.97258
step 1011: Survived [4] steps
Total reward [2.0078083276748657]
Total reward opponent [0]
step 1013: Survived [1] steps
Total reward [-0.19314801692962646]
Total reward opponent [0]
step 1018: Survived [4] steps
T

step 1243: Survived [2] steps
Total reward [0.6276243925094604]
Total reward opponent [0]
step 1244: Survived [0] steps
Total reward [-1.0]
Total reward opponent [0]
step 1247: Survived [2] steps
Total reward [0.4707258939743042]
Total reward opponent [0]
step 1252: Survived [4] steps
Total reward [2.0941836833953857]
Total reward opponent [0]
step 1255: Survived [2] steps
Total reward [0.4603993892669678]
Total reward opponent [0]
step 1258: Survived [2] steps
Total reward [0.49730896949768066]
Total reward opponent [0]
step 1261: Survived [2] steps
Total reward [0.46933674812316895]
Total reward opponent [0]
step 1264: Survived [2] steps
Total reward [0.3688802719116211]
Total reward opponent [0]
step 1266: Survived [1] steps
Total reward [-0.41733527183532715]
Total reward opponent [0]
step 1267: Survived [0] steps
Total reward [-1.0]
Total reward opponent [0]
step 1269: Survived [1] steps
Total reward [-0.21068143844604492]
Total reward opponent [0]
step 1276: Survived [6] steps
To

...

ATTACK step 17847: disconnected 0
step 17848: Survived [3] steps
Total reward [1.4412683248519897]
Total reward opponent [0.8227373361587524]
step 17856: Survived [7] steps
Total reward [4.503973126411438]
Total reward opponent [0]
ATTACK step 17857: disconnected 13
loss = 0.0024910846
ATTACK step 17870: disconnected 8
step 17870: Survived [13] steps
Total reward [11.37525761127472]
Total reward opponent [-0.20047712326049805]
ATTACK step 17885: disconnected 0
step 17889: Survived [18] steps
Total reward [13.695749163627625]
Total reward opponent [0.8200428485870361]
ATTACK step 17900: disconnected 19
step 17903: Survived [13] steps
Total reward [9.361604928970337]
Total reward opponent [0.7625312805175781]
step 17915: Survived [11] steps
Total reward [7.816784024238586]
Total reward opponent [0]
step 17918: Survived [2] steps
Total reward [0.2806987762451172]
Total reward opponent [0]
loss = 0.0041233376
ATTACK step 17924: disconnected 2
step 17931: Survived [12] steps
Total reward [8

step 18526: Survived [5] steps
Total reward [2.8671579360961914]
Total reward opponent [0]
step 18532: Survived [5] steps
Total reward [2.6861053705215454]
Total reward opponent [0]
loss = 0.0007447018
step 18538: Survived [5] steps
Total reward [2.99481463432312]
Total reward opponent [0]
step 18543: Survived [4] steps
Total reward [2.017834424972534]
Total reward opponent [0]
step 18549: Survived [5] steps
Total reward [2.6829845905303955]
Total reward opponent [0]
step 18555: Survived [5] steps
Total reward [2.8128527402877808]
Total reward opponent [0]
step 18561: Survived [5] steps
Total reward [2.7294256687164307]
Total reward opponent [0]
step 18567: Survived [5] steps
Total reward [2.9192187786102295]
Total reward opponent [0]
step 18573: Survived [5] steps
Total reward [3.0205997228622437]
Total reward opponent [0]
step 18579: Survived [5] steps
Total reward [2.7193297147750854]
Total reward opponent [0]
ATTACK step 18580: disconnected 5
step 18585: Survived [5] steps
Total re

ATTACK step 19028: disconnected 14
step 19030: Survived [4] steps
Total reward [2.132327437400818]
Total reward opponent [0.7672491073608398]
ATTACK step 19038: disconnected 3
step 19038: Survived [7] steps
Total reward [6.31377100944519]
Total reward opponent [-1.0]
loss = 0.0033198441
step 19050: Survived [11] steps
Total reward [7.911637902259827]
Total reward opponent [0]
ATTACK step 19051: disconnected 5
ATTACK step 19062: disconnected 9
step 19064: Survived [13] steps
Total reward [9.124065518379211]
Total reward opponent [1.4581565856933594]
ATTACK step 19076: disconnected 15
step 19078: Survived [13] steps
Total reward [9.007124662399292]
Total reward opponent [0.7739920616149902]
ATTACK step 19085: disconnected 2
step 19085: Survived [6] steps
Total reward [5.65505313873291]
Total reward opponent [-1.0]
loss = 0.0019253247
ATTACK step 19097: disconnected 3
ATTACK step 19100: disconnected 18
step 19100: Survived [14] steps
Total reward [12.230036854743958]
Total reward opponent

126 {'disc_lines': array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False]), 'is_illegal': False, 'is_ambiguous': False, 'is_dispatching_illegal': True, 'is_illegal_reco': False, 'opponent_attack_line': None, 'opponent_attack_sub': None, 'opponent_attack_duration': 0, 'exception': [Grid2OpException AmbiguousAction InvalidRedispatching InvalidRedispatching('You cannot ask for a dispatch higher than pmax - pmin  [it would be always invalid because, even if the sepoint is pmin, this dispatch would set it to a number higher than pmax, which is impossible]. Invalid dispatch for generator(s): [1]',)], 'rewards': {}}
step 19466: Survived [52] steps
Total reward [35.41743344068527]
Total reward opponent [2.454618811607361]
step 19472: Survived [5] steps
Total reward [2.9550479650497437]
Total reward opponent [0]
ATTACK step 19487: disconnected 8
loss = 0.0023176663
step 19488: Survived [15]

ATTACK step 20208: disconnected 8
step 20210: Survived [2] steps
Total reward [0.5547575950622559]
Total reward opponent [0.7663624286651611]
loss = 0.0015493416
ATTACK step 20220: disconnected 2
step 20224: Survived [13] steps
Total reward [9.686258554458618]
Total reward opponent [0.8187801837921143]
step 20232: Survived [7] steps
Total reward [4.206245183944702]
Total reward opponent [0]
ATTACK step 20244: disconnected 14
step 20246: Survived [13] steps
Total reward [9.3363938331604]
Total reward opponent [0.8110084533691406]
step 20254: Survived [7] steps
Total reward [4.290820837020874]
Total reward opponent [0]
Successfully saved model at: saved_agent_DDDQN_random_opponent_20000/DDDQN.h5


## Evaluation

In [111]:
nb_episodesode = 10 # number of episodes to evaluate
log_path = './logs-evals'
nb_process = 1 # number of cores to use
max_iter = 150 # maximum number of steps per scenario
verbose = True
save_gif = False

In [114]:
env_name = "rte_case14_realistic"
env = make(env_name, reward_class=L2RPNSandBoxScore,
           other_rewards={
               "reward": L2RPNReward
           })

agent = DoubleDuelingDQN(env.observation_space, env.action_space, name='D3QN', is_training=False)
agent.load('./saved_agent_DDDQN_random_opponent_20000/DDDQN.h5')
    
runner_params = env.get_params_for_runner()
runner_params["verbose"] = False
runner = Runner(**runner_params, agentClass=None, agentInstance=agent)
    
res = runner.run(path_save=log_path, nb_episode=nb_episode, nb_process=nb_process, max_iter=150)
if verbose:
    print("Evaluation summary:")
    for _, chron_name, cum_reward, nb_time_step, max_ts in res:
        msg_tmp = "chronics at: {}".format(chron_name)
        msg_tmp += "\ttotal reward: {:.6f}".format(cum_reward)
        msg_tmp += "\ttime steps: {:.0f}/{:.0f}".format(nb_time_step,
                                                        max_ts)
        print(msg_tmp)

if save_gif:
    save_log_gif(log_path, res)

Agent action size: 141
Successfully loaded network from: ./saved_agent_DDDQN_random_opponent_20000/DDDQN.h5
Evaluation summary:
chronics at: 000	total reward: 134757.593750	time steps: 23/150
chronics at: 001	total reward: 55964.132812	time steps: 16/150
chronics at: 002	total reward: 699308.500000	time steps: 78/150
chronics at: 003	total reward: 97429.289062	time steps: 20/150
chronics at: 004	total reward: 148633.250000	time steps: 24/150
chronics at: 005	total reward: 179308.546875	time steps: 26/150
chronics at: 006	total reward: 20127.648438	time steps: 14/150
chronics at: 007	total reward: 179776.546875	time steps: 26/150
chronics at: 008	total reward: 121001.671875	time steps: 22/150
chronics at: 009	total reward: 1171.987549	time steps: 6/150


NameError: name 'save_gif' is not defined